In [5]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import sklearn
import torch
import torch.nn as nn
%matplotlib inline

# Read image data

In [6]:
image_path = 'SUFRData/image_files/uniform_bg/scaling'

# Torchvision models

In [7]:
import torchvision.models as models
from torchvision.models import VGG

In [8]:
vgg19 = models.vgg19(pretrained=True)

#### Remove last layerfrom network architecture

In [9]:
vgg19.classifier = nn.Sequential(*list(vgg19.classifier.children())[:-1])

### TODO: Double check process image

In [10]:
from torchvision import transforms
from PIL import Image

def load_img(image_path, img_idx):
    return Image.open('{}/{}.jpg'.format(image_path, img_idx))

def process_img(img):    
    transform = transforms.Compose([
     transforms.Resize(256),                    
     transforms.CenterCrop(224),
     transforms.ToTensor(),
     transforms.Normalize(
         mean=[0.485, 0.456, 0.406],
         std=[0.229, 0.224, 0.225]                  
    )])
    img = transform(img)
    processed_img = torch.unsqueeze(img, 0)
    return processed_img

#### Process Data

## Implementation 0: Correlation-based classifier

In [11]:
import os

data = []
for i in range(1,2001):
    if i % 50 == 0:
        print(i)
    img = load_img(image_path, i)
    processed_img = process_img(img)
    data.append(processed_img)
data = torch.cat(data)

50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100
1150
1200
1250
1300
1350
1400
1450
1500
1550
1600
1650
1700
1750
1800
1850
1900
1950
2000


#### Get Train and Test Set 

#### Classifier 

In [13]:
train_face_idxs = np.random.choice(400, 395, replace=False)

mask = np.ones(400, dtype=bool)
mask[train_face_idxs] = False
test_face_idxs = np.arange(400)[mask]
c_offset_values = [-2,-1,1,2]

train_positive_pairs, train_negative_pairs = get_dataset_idxs(train_face_idxs, c_offset_values)
test_positive_pairs, test_negative_pairs = get_dataset_idxs(test_face_idxs, [-1,1])

In [23]:
def cos_sim(a,b):
    dot = np.dot(a, b.T)
    norm_product = np.linalg.norm(a)*np.linalg.norm(b)
    return dot / norm_product

def compare_img_pairs(model, train_idxs, use_corr=False):
    
    similarities = []
    i = 0
    
    for img1_idx, img2_idx in train_idxs:
        img1 = process_img(load_img(image_path, img1_idx))
        img2 = process_img(load_img(image_path, img1_idx))
        vector1 = model(img1).detach().numpy()
        vector2 = model(img2).detach().numpy()
        
        if use_corr:
            similarities.append(np.corrcoef(vector1, vector2)[0][1])
        else:
            similarities.append(cos_sim(vector1, vector2)[0][0])
        print(i)
        i += 1
        
    return similarities

In [24]:
def parse_dataset(relative_path):
    f = open(relative_path, "r")
    dataset_idxs = []
    for line in f.readlines():
        img1_idx, img2_idx, label = line.strip().split(" ")
        dataset_idxs.append(((img1_idx, img2_idx), label))
    return dataset_idxs

def get_negative_pairs(face_idxs, c_offset_values):
    negative_pairs = []
    for c in c_offset_values:
        for i in face_idxs:
            for j in face_idxs:
                # ensures i != j and that there are no duplicate pairs
                if i > j: 
                    pair_idxs = (1+5*i+2,1+5*j+2+c)
                    negative_pairs.append(pair_idxs)
    return negative_pairs

train_face_idxs = np.random.choice(400, 200, replace=False)
c_offset_values = [-1]
num_negative_pairs = train_face_idxs.shape[0] * len(c_offset_values)

def get_dataset_idxs(face_idxs, c_offset_values, num_negative_pairs=None):
    if num_negative_pairs is None:
        num_negative_pairs = face_idxs.shape[0] * len(c_offset_values)
    positive_pairs = np.array([(1+5*i+2,1+5*i+2+c) for c in c_offset_values for i in face_idxs])
    all_negative_pairs = np.array(get_negative_pairs(face_idxs, c_offset_values))
    negative_pairs = all_negative_pairs[list(np.random.choice(len(all_negative_pairs), num_negative_pairs, replace=False)), :]
    return positive_pairs, negative_pairs

In [ ]:
positive_pair_similarities = compare_img_pairs(vgg19, test_positive_pairs, use_corr=False)
negative_pair_similarities = compare_img_pairs(vgg19, test_negative_pairs, use_corr=False)

0
1
2
3


#### AUC Score

In [ ]:
from sklearn.metrics import roc_auc_score

labels = np.concatenate((np.zeros(len(negative_pair_similarities)), np.ones(len(positive_pair_similarities))))
roc_auc_score(labels, negative_pair_similarities + positive_pair_similarities)

#### Taking the correlation of the last layer is not the same as the correlation of the feature layer, I think

In [409]:
W = np.random.rand(4096,1000)
a = np.random.rand(4096)
b = np.random.rand(4096)
print(np.corrcoef(a, b)[0][1])
print(np.corrcoef(np.dot(W.T,a), np.dot(W.T,b))[0][1])
print(cos_sim(a,b))
print(cos_sim(np.dot(W.T,a), np.dot(W.T,b)))

-0.000677816269851673
0.7523936201033044
0.7522615612701068
0.9999721735428134


### Implementation 1: MLP Same/Different Classifier 

In [474]:
vgg_img_idx_to_vec = {}

def get_vgg_vec(img_idx, vgg_img_idx_to_vec, model, image_path):
    if img_idx in vgg_img_idx_to_vec:
        return vgg_img_idx_to_vec[img_idx]
    else:
        img_vec = process_img(load_img(image_path, img_idx))
        img_feature_vec = model.forward(img_vec)
        vgg_img_idx_to_vec[img_idx] = img_feature_vec
        return img_feature_vec

In [4]:
def mlp_preprocessing(model, positive_pairs, negative_pairs):

    pairs = np.concatenate((positive_pairs, negative_pairs))

    x = []
    for img1_idx, img2_idx in pairs:
        img1_feature_vec = get_vgg_vec(img1_idx, vgg_img_idx_to_vec, model, image_path)
        img2_feature_vec = get_vgg_vec(img2_idx, vgg_img_idx_to_vec, model, image_path)
        x_i = torch.cat([img1_feature_vec, img2_feature_vec], dim=1).detach().squeeze().numpy()
        x.append(x_i)

    x = np.array(x)
    y = np.concatenate((np.zeros(len(negative_pairs)), np.ones(len(positive_pairs))))
    
    return x, y

In [3]:
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier(hidden_layer_sizes=[512,64,10])
train_x, train_y = mlp_preprocessing(vgg19, train_positive_pairs, train_negative_pairs)
print(train_x.shape)
print(train_y.shape)

NameError: name 'mlp_preprocessing' is not defined

In [ ]:
model = mlp.fit(train_x,train_y)
model.best_loss_

In [ ]:
test_x, test_y = mlp_preprocessing(vgg19, test_positive_pairs, test_negative_pairs)
model.score(test_x, test_y)

### Implementation 2: MLP Multiclass Classification and Correlation of feature vector for evaluation

In [188]:
mlp = MLPClassifier(hidden_layer_sizes=[512,64,10])

#### Training

In [225]:
zeros = np.zeros((1,400))

i = 1
img_1 = process_img(load_img(image_path, 1))
pretrain_vec_1 = vgg19(img_1)
X = pretrain_vec_1.detach().numpy()

Y = zeros.copy()
Y[:, i // 400] = 1

In [226]:
mlp = MLPClassifier(hidden_layer_sizes=[512,64,10])
model = mlp.fit(X,Y)

In [228]:
from sklearn.neural_network._base import ACTIVATIONS

def deepest_layer(data, MLP, layer=0):
    L = ACTIVATIONS['relu'](np.matmul(data, MLP.coefs_[layer]) + MLP.intercepts_[layer])
    layer += 1
    if layer >= len(MLP.coefs_)-1:
        return L
    else:
        return deepest_layer(L, MLP, layer=layer)

L = deepest_layer(X, model)

#### Evaluation

In [233]:
L

array([[  0.        ,   0.        ,   0.        ,  66.26238368,
         97.19137282, 102.93244527,  89.39741026,   0.        ,
        123.05750886,   0.        ]])